In [1]:
import  pandas  as pd

from tqdm import tqdm

def find_peaks_valleys(GDP_serie):
    GDP = GDP_serie.to_dict()
    years = list(GDP.keys())
    peak_valley = {}
    for y in years:
            try:
                if GDP[y] > GDP[y-1] and GDP[y] > GDP[y+1]:
                    peak_valley[y] = 1;
                elif GDP[y] < GDP[y-1] and GDP[y] < GDP[y+1]:
                    peak_valley[y] = -1;
                else:
                    peak_valley[y] = 0;
            except(KeyError):
                peak_valley[y] = 0
    return peak_valley

def search_cycles(year_start, year_end,GDP_serie,cycle, is_inner = 0):
    GDP = GDP_serie.to_dict()
    found_cycles = 0 
    for y in range(year_start, year_end):
            #if there is at least one valley between y and year_end
        if (GDP[y] > GDP[y+1]) and (y not in cycle.keys()) :
            for i in range(y+1,year_end+1):
                if peaks_valleys[i] == -1:
                    found_cycles = 1
                    last = year_end
                    
                    event = 0
                    for y_t in range(y+1,year_end+1):
                        if GDP[y] <= GDP[y_t] or y_t == year_end:
                            last = y_t
                            event = 1
                            has_inner = search_cycles(y+1,y_t,GDP_serie,cycle,1)
                            break
                            
                    cycle[y] = [last,has_inner,is_inner,event]
                    break
            
    return found_cycles


def add_continuous_pieces(GDP_series_raw,pieces):
    df = GDP_series_raw.fillna(0)
    GDP = df['gdp_pc_KD_var'].values
    if sum(GDP) == 0:
        return 0

    GDP = list(GDP)
    li_index = []
    i = 0
    while i < len(GDP):    
        if (i==0 and GDP[0]!=0):
            li_index.append(0)
        if GDP[i] == 0:
            if i != 0:
                li_index.append(i)
            while(i<len(GDP) and  GDP[i] == 0 ):
                i += 1        
            if i != len(GDP):
                li_index.append(i)
            i -= 1
        i = i+1
    if GDP[i-1] != 0:
        li_index.append(i)
    j = 0
    while 2*j < len(li_index):
        pieces.append(GDP_series_raw.iloc[li_index[2*j]:li_index[2*j+1]])
        j += 1
        
    return 0

df_raw = pd.read_excel('../data/raw/panel_data.xlsx')

df = df_raw[[u'code', u'year','GDPpc_KD ($2000)']]

df.columns = [u'code', u'year','gdp_pc_KD_var']

grouped = df.groupby('code')

all_cycles = {}

for key,group in tqdm(grouped):
    pieces = []
    
    try:
        add_continuous_pieces(group,pieces)
    except(IndexError):
        print('indexError found in')
        print(key)

    cycle = {}
    for piece in pieces:
        piece.index = piece['year']
        peaks_valleys = find_peaks_valleys(piece['gdp_pc_KD_var'])            
        year_range = list(piece.index)
        try:
            search_cycles(year_range[0], year_range[-1], piece['gdp_pc_KD_var'],cycle, is_inner = 0)
        except(KeyError ):
                print('KeyError found in')
                print(key)

    all_cycles[key] = cycle
    
series = []
for key,val in all_cycles.items():
    for k,v in val.items():
        li = [key,k]
        li = li + v
        series.append(li)
        
        

res = pd.DataFrame(series,columns = ['country','year_start','year_end','has_inner','is_inner','event'])


100%|███████████████████████████████████████████████████████████████████████████████| 114/114 [00:00<00:00, 488.74it/s]
